In [1]:
%config InlineBackend.figure_format='png'
%matplotlib inline

In [3]:
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import interp1d

In [4]:
template = Dataset('../create_wrfinput_r/wrfinput_d01_new.nc', 'r+')
met_data = Dataset('../metgrid4wrfinput/met_em.d01.2018-08-01_00:00:00.nc', 'r')

In [5]:
met_data_vars = list(met_data.variables.keys())
for a, b, c, d in zip(
    met_data_vars[::4], met_data_vars[1::4], met_data_vars[2::4], met_data_vars[3::4]
):
    print(("{:<16s}" * 4).format(a, b, c, d))


Times           SOIL_LAYERS     SM              ST              
SM100289        SM028100        SM007028        SM000007        
ST100289        ST028100        ST007028        ST000007        
SNOW            SKINTEMP        RIVERBED        EQZWT           
RECHCLIM        FDEPTH          HARVEST         PLANTING        
SEASON_GDD      CROPTYPE        VAR_SSO         OL4             
OL3             OL2             OL1             OA4             
OA3             OA2             OA1             VAR             
CON             SNOALB          LAI12M          GREENFRAC       
ALBEDO12M       SCB_DOM         SOILCBOT        SCT_DOM         
SOILCTOP        SOILTEMP        HGT_M           LU_INDEX        
LANDUSEF        COSALPHA_V      SINALPHA_V      COSALPHA_U      
SINALPHA_U      XLONG_C         XLAT_C          LANDMASK        
COSALPHA        SINALPHA        F               E               
MAPFAC_UY       MAPFAC_VY       MAPFAC_MY       MAPFAC_UX       
MAPFAC_VX       MAPFAC_MX

In [6]:
template_vars = list(template.variables.keys())
for a, b, c, d in zip(
    template_vars[::4], template_vars[1::4], template_vars[2::4], template_vars[3::4]
):
    print(("{:<16s}" * 4).format(a, b, c, d))

CANWAT          DZS             HGT             ISLTYP          
IVGTYP          LAI             MAPFAC_MX       MAPFAC_MY       
SEAICE          SHDMAX          SHDMIN          SMOIS           
SNOW            TMN             TSK             TSLB            
XLAND           XLAT            XLONG           ZS              


In [7]:
tslb = template.variables['TSLB']
smois = template.variables['SMOIS']
tmn = template.variables['TMN']
tsk = template.variables['TSK']
tsk_met = met_data.variables['SKINTEMP']
st_met = met_data.variables['ST']
sm_met = met_data.variables['SM']

In [8]:
print(tslb[:].dtype)

float32


In [9]:
print(np.ma.count_masked(sm_met[:]))

0


In [10]:
dzs = template.variables['DZS']
zs = template.variables['ZS']
zs_1d = zs[0,:]

In [11]:
print(zs_1d)

[0.05 0.25 0.7  1.5 ]


In [12]:
print(dzs[0,:])

[0.1 0.3 0.6 1. ]


In [13]:
zsi_met = met_data.variables['SOIL_LAYERS']
zsi_met_1d = zsi_met[0,:,0,0]
zsi_met_1d_flipped = zsi_met_1d[::-1]

In [14]:
print(zsi_met_1d_flipped)

[  7.  28. 100. 289.]


In [15]:
nt, nz, ny, nx = tslb.shape
print(nt, nz, ny, nx)

1 4 136 130


In [11]:
st_met_expanded = np.zeros((nt,nz+2, ny, nx), dtype=tslb[:].dtype)

In [13]:
st_met_expanded[0,0,:,:] = tsk_met[0,:,:]
st_met_expanded[0,-1,:,:] = tmn[0,:,:]
st_met_expanded[0,1:-1,:,:] = st_met[0,::-1,:,:]

In [17]:
sm_met_expanded = np.zeros((nt,nz+2, ny, nx), dtype=tslb[:].dtype)

In [15]:
sm_met_expanded[0,0,:,:] = sm_met[0,-1,:,:]
sm_met_expanded[0,-1,:,:] = sm_met[0,0,:,:]
sm_met_expanded[0,1:-1,:,:] = sm_met[0,::-1,:,:]

In [18]:
zsi1 = np.insert(zsi_met_1d_flipped, 0, 0.0)
z_met_expanded = np.zeros(sm_met_expanded.shape[1], dtype = zs.dtype)
z_met_expanded[-1] = 3.0
z_met_expanded[1:-1] = (zsi1[:-1] + zsi1[1:])/2.0/100.0

In [19]:
print(z_met_expanded)

[0.    0.035 0.175 0.64  1.945 3.   ]


In [21]:
f = interp1d(z_met_expanded, st_met_expanded, kind='linear', axis=1)
tslb = f(zs)
del f

In [22]:
f = interp1d(z_met_expanded, sm_met_expanded, kind='linear', axis=1)
smois = f(zs)
del f